## M2.5 - Création d'une analyse reproductible des données climatiques

* Participe: * [** Computational Climate Science **] (https://github.com/openClimatescience/M2-Compuational-climate-science) | ** Leçon précédente ** | ** Next Leçon **

**Contenu:**

- [Création d'un flux de travail reproductible] (# Création-a-REProductible-Workflow)
  - [Notre flux de travail: Téléchargement des données (étape 1)] (# Our-Workflow: -Downing-Téléchargement-the-data- (étape-1))
  - [Notre flux de travail: traitement des données (étape 2)] (# our-workflow: -data-processing- (étape-2))
- [Fichiers d'un projet reproductible] (# a-reProductible-project's-files)
- [Comparaison de plusieurs années de données climatiques] (# Comparaison des années-années-années-climat)
  - [Computing TOA Radiation] (# Computing-ToA-Radiation)
  - [Computing Pet] (# Computing-Pet)
  - [Rééchantillonnage des données des gazouillages] (# Rééchantillonnage-the-chirps-data)
  - [Sélection de la partie d'une série temporelle XARray] (# Sélection de la série-séries à l'heure de la pièce))

---

## Création d'un flux de travail reproductible

Dans la leçon précédente, nous avons utilisé «Dask» et «XArray» pour lire une collection de fichiers NetCDF, puis ** mappé ** A ** Fonction vectorisée ** sur chaque tableau dans une série temporelle. Nous avons produit un graphique qui a montré le rapport précipitation / PET pour Tireret, Algérie, dans les premiers mois de 2024, lors d'une sécheresse sévère.

En soi, le graphique ne nous dit pas à quel point la sécheresse à Tireret est grave. Bien que les précipitations dans la région aient reconstitué moins de 5% de son eau perdue au cours des derniers mois, cela pourrait simplement faire partie du cycle saisonnier normal. En fait, nous savons que janvier à avril est une période relativement humide pour Tireret, mais la question demeure: ** Pouvons-nous comparer cette année aux dernières années? **

Chaque fois que nous voulons appliquer une analyse terminée à un nouvel ensemble de données, au fil du temps ou de l'espace, c'est une opportunité pour nous d'améliorer la façon dont notre flux de travail est représenté. Considérez les deux scripts ci-dessous, qui représentent les deux premières étapes de notre flux de travail.

### Notre flux de travail: téléchargement des données (étape 1)

Le premier fichier peut être nommé quelque chose comme ** `yyyymmdd_step1_download_merra2_data.py` **. N'oubliez pas que `yyyymmdd` est la date d'aujourd'hui, et cela nous aidera à ** lier nos fichiers de sortie avec le code qui les a créés. **

Notez que les fichiers Python, avec l'extension de fichier `* .py`, peuvent avoir un docstring de niveau de fichier **, ** qui, dans l'exemple ci-dessous, est la chaîne multi-ligne Python commençant par` '' ''. Les docstrings au niveau du fichier doivent commencer sur la première ligne d'un fichier. Ils sont extrêmement importants pour les flux de travail reproductibles; La première ligne d'un fichier est le premier endroit où vous chercherez à comprendre quel est le but du fichier!

`` Python
'' '
Télécharge les données Merra-2 M2SDNXSLV, pour les 5 premiers mois d'une année.
En savoir plus sur Merra-2 ici:

    https://gmao.gsfc.nasa.gov/reanalysis/merra-2/

Les données sont téléchargées dans ce dossier:

    data_raw / Merra2
'' '

importer la terre

Data_year = 2023

auth = EarthAccess.login ()

résultats = EarthAccess.search_data (
    short_name = 'm2sdnxslv',
    temporal = (f "{data_year} -01-01", f "{data_year} -05-31"))

# Pourrait prendre environ 1 minute sur une connexion à large bande
EarthAccess.Download (résultats, «data_raw / merra2»)
`` '

& # x1F449; ** De haut en bas, notez que: **

- Nous avons un ** Docstring de niveau de fichier ** en haut du script avec des informations importantes sur le but du script, où obtenir plus d'informations et comment le script modifie notre système de fichiers.
- Toutes nos instructions «Import» sont près du haut du script. Cela signale à quelqu'un qui lise notre script quels modules Python sont nécessaires pour exécuter le script. Nous ne voulons pas mettre des instructions «import» plus loin dans le script car il serait plus difficile de les trouver. Cela pourrait conduire à une surprise «Importorror» lors de l'exécution du script.
- Les paramètres qui pourraient être modifiés lors de l'exécution du script sont clairement identifiés, en utilisant toutes les lettres majuscules pour définir la variable, près du haut du script. Par exemple, `data_year` est une variable que nous pourrions vouloir modifier lors de l'exécution du script plusieurs fois. Le mettre en haut de notre script, en utilisant toutes les lettres majuscules, aide à éviter la difficulté de lire chaque ligne du script pour trouver la pièce qui doit changer.

### Notre flux de travail: traitement des données (étape 2)

L'étape suivante consiste à lire les fichiers de données et à calculer le rayonnement haut de gamme (TOA). Le deuxième fichier peut être nommé ** `yyyymmdd_step2_compute_toa_radiation.py` **.

`` Python
'' '
Calcule le rayonnement haut de l'atmosphère (TOA) d'une série de Merra-2 
Les fichiers M2SDNXSLV, ensuite écrit un fichier NetCDF de sortie. Le rayonnement TOA est
calculé selon la formule FAO pour le rayonnement extraterrestre:

    https://www.fao.org/4/x0490e/x0490e07.htm#radiation
'' '

Importer Numpy comme NP
importer xarray comme xr

# Remarque: ce sera différent sur votre système informatique et vous devriez
# Utilisez un chemin absolu, pas un chemin relatif
Merra2_data_dir = './data_raw/merra2'
Data_year = 2023
Output_file = f './ sorties / yyyymmdd_merra2_ {data_year} _with_toa-radiation.nc'

def main ():
    ds = xr.open_mfdataset (f '{Merra2_data_dir} / * {data_year} *. nc4', chunks = 'auto')
    lats = ds ['lat']. valeurs.Reshape ((361, 1, 1)) \
        .repeat (ds.lon.size, axe = 1) \
        .repeat (ds.time.size, axe = 2)
    ds ['lat_grid'] = (('lat', 'lon', 'temps'), lats)

    # Calculer le rayonnement TOA
    template = ds ['t2mmean']
    template.name = 'toa_radiation'
    result = xr.map_blocks (toa_radiation_wrapper, ds, modèle = modèle)
    TOA_RESULT = Result.Compute ()
    # Convertir le rayonnement TOA de [MJ M-2 Day-1] en [MM H2O Day-1]
    ds ['toa_radiation'] = toa_result * 0,408
    
    # Écrivez le fichier sur disque, juste les variables importantes
    ds = ds [['t2mmax', 't2mmean', 't2mmin', 'toa_radiation']]]
    comp = dict (zlib = true, complevel = 5)
    codage = {var: comp pour var dans ds.data_vars}
    ds.to_netcdf (output_file, format = 'netcdf4', coding = coding)

    
def toa_radiation (latitude, doy):
    '' '
    Rayonnement haut de l'atmosphère (TOA) pour une latitude (l) et un jour de l'année donnée
    (Doy) peut être calculé comme:

    R = ((24 * 60) / pi) * g * d * (w * sin (l) * sin (d) + cos (l) * cos (d) * sin (w))

    Où g est la constante solaire, 0,0820 [MJ M-2 Day-1]; D est le Sun-Sun
    distance; W est l'angle de l'heure du coucher du soleil; et d est l'angle de déclinaison solaire.
    
    Pour plus d'informations, consultez la documentation de la FAO:

        https://www.fao.org/4/x0490e/x0490e07.htm#radiation
    
    Paramètres
    ----------
    Latitude: flotter
        La latitude sur terre, en degrés
    doy: int
        Le jour de l'année (Doy), un entier sur [1 366]
    
    Rendements
    -------
    Nombre
        Rayonnement haut de l'atmosphère (TOA), dans [MJ M-2 Day-1]
    '' '
    Solar_Constant = 0.0820 # [MJ M-2 Day-1]
    pi = 3,14159
    
    # Convertir la latitude des degrés en radians
    lat_radians = np.deg2rad (latitude)
    # Distance terrestre, en fonction du jour de l'année (DOY)
    Earth_Sun_dist = 1 + 0,0033 * np.cos (doy * ((2 * pi) / 365))
    # Déclinaison solaire, en fonction de Doy
    déclinaison = 0,409 * np.sin (doy * ((2 * pi) / 365) - 1,39)
    
    # Angle d'heure au coucher du soleil; Nous utilisons np.where () ci-dessous pour garder contre
    # avertissements où arccos () renvoie des valeurs non valides, qui
    # se produit lorsque l'argument est à l'extérieur [-1, 1]
    _hour_angle = -np.tan (lat_radians) * np.tan (déclinaison)
    _hour_angle = np.where (np.abs (_hour_angle)> 1, np.nan, _hour_angle)
    Sunset_hour_angle = np.arccos (_hour_angle)
    
    return ((24 * 60) / pi) * Solar_Constant * Earth_Sun_dist * \
        (Sunset_hour_angle * np.sin (lat_radians) * np.sin (déclinaison) +
            np.cos (lat_radians) * np.cos (déclinaison) * np.sin (Sunset_hour_angle))


def toa_radiation_wrapper (ensemble de données):
    '' '
    Enveloppe TOA_RADIATION pour travailler avec un XArray.Dataset

    Paramètres
    ----------
    ensemble de données: xArray.Dataset
        Ensemble de données d'entrée avec des variables "lat_grid" et "time"

    Rendements
    -------
    xarray.dataarray
    '' '
    Renvoie TOA_RADIATION (ensemble de données ['lat_grid'], ensemble de données ['Time.Dayofyear'])


# Si le fichier est exécuté en tant que script, exécutez la fonction Main ()
Si __Name__ == '__MAIN__':
    principal()
`` '

& # x1F449; ** Encore une fois, notez que: **

- Les instructions de docstring ** au niveau du fichier ** et «Import» près du haut du script aident les utilisateurs à identifier l'objectif du script et quels modules Python sont nécessaires pour l'exécuter.
- Notre fonction `toa_radiation ()` a également un docstring au niveau de la fonction ** qui décrit comment fonctionne la fonction: ce que ** arguments d'entrée ** il nécessite et quelle est la valeur de retour ** **.

& # x1F449; ** Considérez la ligne qui contient `si __name__ == '__main __' '; Qu'est-ce que cela signifie?**

- Chaque fichier Python, ou fichier `* .py`, a un code qui peut être exécuté de deux manières, soit en exécutant` Python myscript.py` (en tant que script) ou en important * le fichier en tant que module, par exemple, «Importer myscript».
- Chaque fichier Python, lorsque le code qu'il contient est exécuté, introduit une variable appelée `__name__` qui indique le nom du module. Lorsqu'un fichier Python est exécuté en tant que script, alors `__name__` est défini égal à` '__main __' '. Par conséquent, `__name__` est une variable que nous pouvons utiliser pour tester si le code Python est actuellement exécuté en tant que script ou s'il a été importé en tant que module.

** Pourquoi nous soucions-nous de savoir si le fichier est exécuté en tant que script ou s'il a été importé comme un module? ** Lorsqu'un fichier Python est importé en tant que module, tout le code de ce fichier est exécuté. Cela signifie que tout code python qui est en dehors d'une ** définition de fonction ** sera exécuté à chaque fois que nous importons le module, ce qui n'est probablement pas ce que nous voulons, surtout si le fichier contient des fonctions utiles comme `Toa_radiation ()` que nous Pourrait vouloir ** réutiliser ** ailleurs; Autrement dit, nous pourrions vouloir écrire quelque chose comme «à partir de MyScript Import Toa_Radiation ()» dans un autre script.

Le code que nous voulons exécuter * uniquement lorsque le fichier est exécuté en tant que script * doit être placé dans une fonction comme `main () ', qui peut être appelé conditionnellement:
`` Python
# Si le fichier est exécuté en tant que script, exécutez la fonction Main ()
Si __Name__ == '__MAIN__':
    principal()
`` '

Si cela est déroutant, pour l'instant, considérez simplement l'instruction `if` ci-dessus comme une technique Python" magique "qui nous permet d'écrire des fichiers de code Python qui peuvent à la fois être exécutés sous forme de scripts et importés sous forme de modules.

---

## Les fichiers d'un projet reproductible

Il y a d'autres étapes dans notre analyse, mais nous vous laissons le peu comme un exercice pour écrire des scripts réutilisables et bien documentés comme celui ci-dessus. Votre répertoire de projet final peut ressembler à ceci:

! [] (./ actifs / m2_file_tree_workflow.png)

& # x1F449; ** Notez que chacun des fichiers «scripts» comprend les mots «Step1», «Step2» ou «Step3», indiquant l'ordre dans lequel le workflow doit être exercé. doit être exécuté; Cela pourrait même vous aider si vous revenez à ce projet des mois plus tard et que vous ne vous souvenez pas de ce que vous avez fait auparavant!

---

## Comparaison de plusieurs années de données climatiques

En fin de compte, nous voulons répondre à notre question de la dernière leçon: comment le rapport précipitation / périphérique pour l'hiver 2024 se compare-t-il à celui de l'hiver 2023 à Tireret?

Notre flux de travail reproductible, ci-dessus, pourrait être utilisé pour répondre à cette question. Voyons également comment répondre à la question dans une session Python ** interactive, ** c'est-à-dire ce cahier de jupyter. Si nous avons déjà téléchargé les données Merra-2 pour 2023 et 2024, nous sommes prêts à charger les données à l'aide de `xarray.open_mfdataset () '.

In [ ]:
import xarray as xr
from matplotlib import pyplot

ds_chirps = xr.open_mfdataset('data_raw/CHIRPS/CHIRPS-v2_Africa_monthly_2014-2024.nc')
ds_merra2 = xr.open_mfdataset('data_raw/MERRA2/*.nc4', chunks = 'auto')

Notre script Python, `yyyymmdd_step2_compute_toa_radiation.py`, contient une fonction utile et réutilisable,` toa_radiation () `. ** Comment pouvons-nous utiliser cette fonction sans copier et coller la déclaration de fonction dans un autre script, ou dans ce cahier? ** En général, nous devons éviter d'avoir plus d'une copie de n'importe quelle fonction. L'une des principales raisons d'écrire une fonction est de ne pas avoir à écrire le même code deux fois!

& # x1F449; Comme suggéré ci-dessus, lorsque nous avons inclus l'instruction `if __name__ == '__main __':` dans notre script, détecter le nom du module avec `__name__` est un moyen utile de déterminer si le code Python est exécuté en tant que script ou en tant que module. Tant que nous pouvons voir le script nommé `yyymmmdd_step2_compute_toa_radiation.py` quelque part dans notre arborescence de fichiers, nous devrions pouvoir l'importer comme module. Ci-dessous, nous utilisons la bibliothèque `OS` pour examiner les fichiers du répertoire actuel. Nous pouvons voir qu'il y a un répertoire «scripts» en dessous de celui-ci ...

In [ ]:
import os

# Display the files in the current directory
files = os.listdir('.') # The single dot "." indicates the current directory
files.sort()
files

Et à l'intérieur de «scripts» se trouve le script Python que nous voulons importer en tant que module!

In [ ]:
os.listdir('scripts')

Comment faisons-nous cela? Étant donné que les répertoires et les fichiers Python `* .py` peuvent être traités comme si ce sont des modules, tout ce que nous devons faire est d'écrire une instruction` IMPORT 'du module comme celle ci-dessous.

In [ ]:
from scripts.YYYYMMDD_step2_compute_TOA_radiation import toa_radiation_wrapper

Notez que nous venons d'importer un nom de variable de ce script, `toa_radiation_wrapper ()` fonction, ce qui nous donne accès à la fonction `toa_radiation ()`. Et parce que nous avons écrit un docstring au niveau de la fonction aussi utile ** plus tôt, nous pouvons voir toutes les informations sur la façon d'utiliser cette fonction en appelant `` help () 'sur notre fonction.

In [ ]:
help(toa_radiation_wrapper)

### Radiation TOA informatique

Comme précédemment, afin de calculer le rayonnement TOA, nous voulons créer une grille de latitude afin que nous puissions ** vectoriser ** notre étape `toa_radiation () '.

In [ ]:
lat_grid = ds_merra2.lat.values.reshape(1, 361, 1)\
    .repeat(ds_merra2.lon.size, axis = 2)\
    .repeat(ds_merra2.time.size, axis = 0)
lat_grid.shape

In [ ]:
ds_merra2['lat_grid'] = (('time', 'lat', 'lon'), lat_grid)
ds_merra2

Et nous avons besoin du jour de l'année, mais cela est déjà disponible en tant que `DS_MERRA2 ['Time.Dayofyear']`.

Par conséquent, nous sommes prêts à calculer le rayonnement TOA!

In [ ]:
# Converting TOA Radiation from [MJ m-2 day-1] to [mm H2O day-1]
toa_rad = toa_radiation_wrapper(ds_merra2) * 0.408
toa_rad.sel(time = '2024-05-01').plot()

### Computing Pet

Et nous irons également et sous-ensemble les données Merra-2 à Tireret également.

In [ ]:
toa_rad.shape

In [ ]:
import numpy as np

ds_merra2['toa_radiation'] = toa_rad

# Select just the Tiaret region
merra2_tiaret = ds_merra2.sel(lon = -1.32, lat = 35.37, method = 'nearest')

# Compute PET using the Hargreaves equation
merra2_tiaret_pet = 0.0023 * merra2_tiaret['toa_radiation'] * np.sqrt(
    merra2_tiaret['T2MMAX'] - merra2_tiaret['T2MMIN']) * (merra2_tiaret['T2MMEAN'] + 17.8)

### rééchantillonnant les données de gazouillis

Enfin, nous devons traiter les données mensuelles de précipitations mensuelles sur les précipitations quotidiennes, pour la zone Tireret, comme nous l'avons fait dans la dernière leçon.

In [ ]:
chirps_tiaret = ds_chirps['precip'].sel(x = slice(0.8, 1.8), y = slice(36.1, 35.1))
chirps_tiaret_daily = chirps_tiaret.resample(time = 'D').nearest().mean(['x', 'y']) / 30
chirps_tiaret_daily

### Sélection de la partie d'une série chronologique de `XArray '

Nous sommes maintenant prêts à comparer 2023 et 2024.

En supposant que notre répertoire `data_raw / merra2` contient les données que nous avons téléchargées à partir de 2023 et 2024, alors notre ensemble de données` DS_MERRA2` contient des données à partir de 2023 et 2024. Yyyy-mm-dd ') `modèle, mais comment pouvons-nous sélectionner toutes les données d'une année, d'un mois ou d'une gamme de dates spécifiques?

Chaque dimension `xarray» qui a le type `DateTime64 [ns] 'également un` DatetimeAccessor', accessible via la propriété `DT '.

In [ ]:
merra2_tiaret_pet.time.dt

[La propriété `dt` peut être utilisée pour sous-ensemble une série temporelle.] (Https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.datetimeaccessor.html)

In [ ]:
merra2_tiaret_pet.time.dt.year

Par exemple, pour sélectionner uniquement les données de 2023 ...

In [ ]:
# Select only those data points along the time access where time.year is in a list of years
merra2_tiaret_pet.sel(time = merra2_tiaret_pet.time.dt.year.isin([2023]))

In [ ]:
ratios = []
precip = []
for year in [2023, 2024]:
    pet_this_year = merra2_tiaret_pet.sel(time = merra2_tiaret_pet.time.dt.year.isin([year]))
    precip_this_year = chirps_tiaret_daily.sel(time = chirps_tiaret_daily.time.dt.year.isin([year]))
    # Select the first 122 days
    pet_this_year = pet_this_year.isel(time = slice(0, 122))
    precip_this_year = precip_this_year.isel(time = slice(0, 122))
    # NOTE: We must use the .values attribute because these are from two different datasets
    ratios.append(100 * precip_this_year.values / pet_this_year.values)
    precip.append(precip_this_year.values)

In [ ]:
pyplot.figure(figsize = (12, 5))
for y, year in enumerate([2023, 2024]):
    pyplot.plot(pet_this_year.time, ratios[y], label = year)
    
pyplot.ylabel('Precipitation-to-PET Ratio (mm day-1)')
pyplot.legend()

** L'intrigue ci-dessus suggère que les conditions hydrologiques au début de 2024 ne sont pas si différentes de l'année précédente. ** Pourquoi ces conditions de saisie sèches ont-elles produit une sécheresse socioéconomique ** en 2024 mais pas en 2023? Comme nous l'avons vu lorsque nous avons tracé les ratios de précipitation à long terme, en utilisant des données sur la terre cuite, les précipitations de la saison humide sont inférieures à celles de plusieurs années. Une explication de la sécheresse socioéconomique de 2024 pourrait être que l'effet * cumulatif * de plusieurs années de faibles précipitations a créé des conditions de sécheresse en 2024 qui n'étaient pas aussi apparentes en 2023.

Nous pouvons visualiser à nouveau la baisse à long terme des précipitations hivernales à l'aide des données de gazouillis. Malgré une augmentation relative des précipitations en 2022, il est évident que les précipitations hivernales sont inhabituellement faibles depuis 2019.

In [ ]:
chirps_tiaret.sel(time = chirps_tiaret.time.dt.month.isin([1,2,3,4,5])).groupby('time.year').sum().mean(['x', 'y']).plot()
pyplot.ylabel('January - May Precipitation (mm)')

---

## Plus de ressources

- [`xarray.core.accessor_dt.datetimeAccessor` (documentation)] (https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.datetetimeaccessor.html)